In [ ]:
# Import Libraries

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization

## **1. Data Augmentation**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 45,
                                   horizontal_flip=True,
                                   shear_range = 0.3,
                                   zoom_range = 0.3)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Let's create a Training & Test set with an augmented images

train_set = train_datagen.flow_from_directory('/content/drive/MyDrive/AI Projects/Image Classification on Custom Dataset using Transfer Learning/train',
                                              target_size = (120, 120), 
                                              class_mode = 'binary',batch_size = 64)


test_set = train_datagen.flow_from_directory('/content/drive/MyDrive/AI Projects/Image Classification on Custom Dataset using Transfer Learning/test',
                                              target_size = (120, 120), 
                                              class_mode = 'binary',
                                              batch_size = 64)

Found 2011 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## **2. Build the Model**

In [ ]:
model = Sequential()

# 1st Convolution Block
model.add(Conv2D(filters = 64, kernel_size = (3, 3), input_shape = (120, 120, 3), activation = 'relu'))  # 1st Convolution Layer
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))   # 2nd Convolution Layer
model.add(MaxPool2D(pool_size=(2,2)))   # Pooling Layer

# 2nd Convolution Block
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'))  # 3rd Convolution Layer
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'))   # 4th Convolution Layer
model.add(MaxPool2D(pool_size=(2,2)))


# Flattening Layer
model.add(Flatten())


# Fully Connected layer
model.add(Dense(64, activation = 'relu', kernel_initializer='he_normal'))    # Hidden Layer 1
model.add(Dropout(0.5))

model.add(Dense(32, activation = 'relu', kernel_initializer='he_normal'))     # Hidden Layer 2
model.add(Dropout(0.3))

# Adding the Output layer
model.add(Dense(3, activation = 'softmax'))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 118, 118, 64)      1792      
                                                                 
 conv2d_9 (Conv2D)           (None, 116, 116, 64)      36928     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 58, 58, 64)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 56, 56, 128)       73856     
                                                                 
 conv2d_11 (Conv2D)          (None, 54, 54, 128)       147584    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 27, 27, 128)      0         
 2D)                                                  

In [ ]:
# Compile the Model
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer='adam')

In [ ]:
# Train the Model
history = model.fit(train_set, batch_size= 64, epochs = 10, validation_data = test_set)

NameError: ignored

## **3. Model Evaluation**

In [ ]:
score = model.evaluate(test_set)
print('Test Loss: ', score[0])
print('Test accuracy: ', score[1])

16/16 [==============================] - 8s 499ms/step - loss: 0.6932 - accuracy: 0.5000
Test Loss:  0.6931527256965637
Test accuracy:  0.5


## **VGG - 19**

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
vgg19 = VGG19(input_shape = (120, 120, 3), weights = 'imagenet', include_top = False)

80134624/80134624 [==============================] - 5s 0us/step


In [ ]:
# Don't train the existing weights

for layer in vgg19.layers:
  layer.trainable = False

In [ ]:
# Fully Connected Layers

flatten = Flatten()(vgg19.output)
dense = Dense(128, activation = 'relu')(flatten)
dense = Dense(64, activation = 'relu')(dense)
prediction = Dense(3, activation = 'softmax')(dense)

In [ ]:
model = Model(inputs = vgg19.input, outputs = prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 120, 120, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 120, 120, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 120, 120, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 60, 60, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 60, 60, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 60, 60, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 30, 30, 128)       0     

In [ ]:
# Compile the Model
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer='adam')

In [ ]:
# Train the Model
history = model.fit(train_set, batch_size= 64, epochs = 10, validation_data = test_set)

Epoch 1/10
32/32 [==============================] - 32s 880ms/step - loss: 0.6078 - accuracy: 0.6693 - val_loss: 0.4952 - val_accuracy: 0.7680
Epoch 2/10
32/32 [==============================] - 24s 765ms/step - loss: 0.4580 - accuracy: 0.7921 - val_loss: 0.4140 - val_accuracy: 0.8000
Epoch 3/10
32/32 [==============================] - 26s 821ms/step - loss: 0.4307 - accuracy: 0.7916 - val_loss: 0.4176 - val_accuracy: 0.7990
Epoch 4/10
32/32 [==============================] - 26s 820ms/step - loss: 0.3845 - accuracy: 0.8225 - val_loss: 0.4403 - val_accuracy: 0.7970
Epoch 5/10
32/32 [==============================] - 24s 764ms/step - loss: 0.3723 - accuracy: 0.8230 - val_loss: 0.4049 - val_accuracy: 0.8320
Epoch 6/10
32/32 [==============================] - 23s 732ms/step - loss: 0.3680 - accuracy: 0.8359 - val_loss: 0.3907 - val_accuracy: 0.8280
Epoch 7/10
32/32 [==============================] - 24s 746ms/step - loss: 0.3633 - accuracy: 0.8354 - val_loss: 0.3637 - val_accuracy: 0.8430

In [ ]:
score = model.evaluate(test_set)
print('Test Loss: ', score[0])
print('Test accuracy: ', score[1])

16/16 [==============================] - 8s 520ms/step - loss: 0.3827 - accuracy: 0.8290
Test Loss:  0.38273242115974426
Test accuracy:  0.8289999961853027


## **ResNet-50**

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
resnet = ResNet50(input_shape = (120, 120, 3), weights = 'imagenet', include_top = False)

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
# Don't train the existing weights

for layer in resnet.layers:
  layer.trainable = False

In [ ]:
# Fully Connected Layers

flatten = Flatten()(resnet.output)
dense = Dense(128, activation = 'relu')(flatten)
dense = Dense(64, activation = 'relu')(dense)
prediction = Dense(3, activation = 'softmax')(dense)

In [ ]:
model = Model(inputs = resnet.input, outputs = prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 120, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 126, 126, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 60, 60, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 60, 60, 64)   256         ['conv1_conv[0][0]']         

In [ ]:
# Compile the Model
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer='adam')

In [ ]:
# Train the Model
history = model.fit(train_set, batch_size= 64, epochs = 10, validation_data = test_set)

Epoch 1/10
32/32 [==============================] - 1275s 40s/step - loss: 0.7548 - accuracy: 0.5017 - val_loss: 0.6895 - val_accuracy: 0.5290
Epoch 2/10
32/32 [==============================] - 25s 776ms/step - loss: 0.6949 - accuracy: 0.5316 - val_loss: 0.6863 - val_accuracy: 0.5410
Epoch 3/10
32/32 [==============================] - 25s 775ms/step - loss: 0.6862 - accuracy: 0.5549 - val_loss: 0.6724 - val_accuracy: 0.6030
Epoch 4/10
32/32 [==============================] - 25s 775ms/step - loss: 0.6832 - accuracy: 0.5564 - val_loss: 0.6687 - val_accuracy: 0.5950
Epoch 5/10
32/32 [==============================] - 25s 781ms/step - loss: 0.6695 - accuracy: 0.5967 - val_loss: 0.6697 - val_accuracy: 0.6020
Epoch 6/10
32/32 [==============================] - 25s 780ms/step - loss: 0.6622 - accuracy: 0.5992 - val_loss: 0.6592 - val_accuracy: 0.6120
Epoch 7/10
32/32 [==============================] - 26s 832ms/step - loss: 0.6641 - accuracy: 0.5937 - val_loss: 0.6869 - val_accuracy: 0.5450

## **Inception Net V3**

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
inception = InceptionV3(input_shape = (120, 120, 3), weights = 'imagenet', include_top = False)

In [ ]:
# Don't train the existing weights

for layer in inception.layers:
  layer.trainable = False

In [ ]:
# Fully Connected Layers

flatten = Flatten()(inception.output)
dense = Dense(128, activation = 'relu')(flatten)
dense = Dense(64, activation = 'relu')(dense)
prediction = Dense(3, activation = 'softmax')(dense)

In [ ]:
model_inception = Model(inputs = inception.input, outputs = prediction)

In [ ]:
model_inception.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 120, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 59, 59, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 59, 59, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [ ]:
# Compile the Model
model_inception.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer='adam')

In [ ]:
# Train the Model
history = model_inception.fit(train_set, batch_size= 64, epochs = 10, validation_data = test_set)

Epoch 1/10
32/32 [==============================] - 1280s 40s/step - loss: 1.5555 - accuracy: 0.7489 - val_loss: 0.3171 - val_accuracy: 0.8620
Epoch 2/10
32/32 [==============================] - 22s 687ms/step - loss: 0.3521 - accuracy: 0.8543 - val_loss: 0.2942 - val_accuracy: 0.8670
Epoch 3/10
32/32 [==============================] - 23s 722ms/step - loss: 0.3117 - accuracy: 0.8657 - val_loss: 0.3162 - val_accuracy: 0.8640
Epoch 4/10
32/32 [==============================] - 23s 736ms/step - loss: 0.3008 - accuracy: 0.8593 - val_loss: 0.3044 - val_accuracy: 0.8660
Epoch 5/10
32/32 [==============================] - 23s 710ms/step - loss: 0.2673 - accuracy: 0.8866 - val_loss: 0.2841 - val_accuracy: 0.8680
Epoch 6/10
32/32 [==============================] - 25s 779ms/step - loss: 0.2452 - accuracy: 0.8921 - val_loss: 0.2430 - val_accuracy: 0.8950
Epoch 7/10
32/32 [==============================] - 23s 724ms/step - loss: 0.2746 - accuracy: 0.8881 - val_loss: 0.2626 - val_accuracy: 0.8960

## Evaluation

In [ ]:
score = model_inception.evaluate(test_set)
print('Test Loss: ', score[0])
print('Test accuracy: ', score[1])

16/16 [==============================] - 8s 482ms/step - loss: 0.2665 - accuracy: 0.8850
Test Loss:  0.26654818654060364
Test accuracy:  0.8849999904632568


## **Make a Prediction System**

In [ ]:
# Save the Model
model_inception.save("/content/drive/MyDrive/AI Projects/Image Classification on Custom Dataset using Transfer Learning/Multi_Class_Image_Classification_System.h5")

In [ ]:
from keras.utils import load_img, img_to_array

test_image = load_img('/content/drive/MyDrive/AI Projects/Image Classification on Custom Dataset using Transfer Learning/Sample Images/pexels-pixabay-45201.jpg', target_size=(120, 120))
test_image = img_to_array(test_image)
test_image =np.expand_dims(test_image, axis = 0)

result = model_inception.predict(test_image)

if result[0][0] == 0:
  prediction = 'Cat'
  print(prediction)

elif result[0][0] == 1:
  prediction = 'Dog'
  print(prediction)
else:
  prediction = 'Panda'
  print(prediction)

1/1 [==============================] - 0s 25ms/step
Dog
